In [18]:
%%time

import nltk
import numpy
import time
import random
import unidecode

import pandas as pd

from nltk.corpus import stopwords
from numba import njit, jit, cuda
import numpy as np

nltk.download('stopwords')

%load_ext line_profiler


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
CPU times: user 29.1 ms, sys: 0 ns, total: 29.1 ms
Wall time: 27.5 ms


[nltk_data] Downloading package stopwords to /home/tmtoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
%%time

df = pd.read_csv("news_dataset.csv")
# df = df[:10000]

CPU times: user 3.97 s, sys: 456 ms, total: 4.42 s
Wall time: 4.42 s


In [3]:
%%time

# remove accents, special characters and lowercase
# https://stackoverflow.com/questions/37926248/how-to-remove-accents-from-values-in-columns
df.content = df.content.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.replace('[^a-z ]', '')
print(df.content.head())

# remove stop words
# https://stackoverflow.com/questions/29523254/python-remove-stop-words-from-pandas-dataframe
stopwords = stopwords.words('english')
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
df['content'] = df['content'].str.replace(pat, '')


# remove multiple spaces in a
df['content'] = df['content'].str.replace(r'\s+', ' ')
df = df[df['content'].notna()]
print(df.content.head())


0    washington eeaaaaeeeace    congressional repub...
1    after the bullet shells get counted the blood ...
2    when walt disneys bambi opened in  critics pra...
3    death may be the great equalizer but it isnt n...
4    seoul south korea     north koreas leader kim ...
Name: content, dtype: object
0    washington eeaaaaeeeace congressional republic...
1     bullet shells get counted blood dries votive ...
2     walt disneys bambi opened critics praised spa...
3    death may great equalizer isnt necessarily eve...
4    seoul south korea north koreas leader kim said...
Name: content, dtype: object
CPU times: user 2min 47s, sys: 800 ms, total: 2min 48s
Wall time: 2min 48s


In [4]:
%%time

vocab_set = set()

def compute_dict(input):
    result = dict()

    for word in input.split(' '):
        if word != '':
            vocab_set.add(word)
            if word in result:
                result[word] += 1
            else:
                result[word] = 1
    return result

df['content_dict'] = df['content'].apply(lambda content: compute_dict(content))
vocab = list(vocab_set)

tmp ={}
count = 0
for word in vocab:
    tmp[word] = count
    count += 1
vocab = tmp

CPU times: user 20.5 s, sys: 820 ms, total: 21.4 s
Wall time: 21.4 s


In [5]:
%%time

#Toevoegen van extra column met index ineens
def remove_dupes(input):
    output = list(set(input.split(' ')))
    if '' in output:
        output.remove('')
    for i, value in enumerate(output):
        output[i] = vocab[value]
    return np.array(output)
df['content_list'] = df['content'].apply(lambda content: remove_dupes(content))

CPU times: user 20.5 s, sys: 168 ms, total: 20.7 s
Wall time: 20.7 s


In [6]:
%%time

numberOfTopics = 20
topics = [i for i in range(0, numberOfTopics)]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


In [7]:
%%time

wordToTopic = numpy.zeros((len(vocab), numberOfTopics), dtype=numpy.int64)
documentToTopic = numpy.zeros((len(df.content), numberOfTopics))
topicAssignment = []
words_in_topic = numpy.zeros(numberOfTopics, dtype=numpy.int64)
words_in_doc = [0 for i in range(len(df.content))]
for i, content in enumerate(df.content_list):
    tmp = []
    for word in content:
        topic = numpy.random.choice(topics)
        tmp.append(topic)
        wordToTopic[word, topic] += 1
        documentToTopic[i, topic] += 1
        words_in_topic[topic] += 1
        words_in_doc[i] += 1
    topicAssignment.append(np.array(tmp))
    
topicAssignment = np.array(topicAssignment)

CPU times: user 6min 6s, sys: 220 ms, total: 6min 6s
Wall time: 6min 6s


In [8]:
%%time

alpha = 1
beta = 0.001

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


In [73]:
%%time

@njit()
def calc_fast(TA_d, doc_index, word_index, wordToTopic, word, words_in_topic, total_unique_word_b, document_words, WID):
    document_words[TA_d[word_index]] -= 1
    wordToTopic[word, TA_d[word_index]] -= 1
    words_in_topic[TA_d[word_index]] -= 1
                
    new_topic = numpy.argmax(np.multiply(np.divide((wordToTopic[word] + beta), (words_in_topic + total_unique_word_b)), np.divide((document_words + alpha), (WID))))

    TA_d[word_index] = new_topic
    document_words[new_topic] += 1
    wordToTopic[word, new_topic] += 1
    words_in_topic[new_topic] += 1
    
def gibs_it_sanic(i):
    total_unique_word_b = len(vocab) * beta
    topic_count_a = numberOfTopics * alpha
    for iteration in range(i):
        print("Iteration:", iteration + 1)
        
        for doc_index, word_list in enumerate(df.content_list):
            document_words = documentToTopic[doc_index]
            WID = words_in_doc[doc_index] - 1 + topic_count_a
            TA_d = topicAssignment[doc_index]
            
            for word_index, word in enumerate(word_list):
                calc_fast(TA_d, doc_index, word_index, wordToTopic, word, words_in_topic, total_unique_word_b, document_words, WID)

                


CPU times: user 0 ns, sys: 1.98 ms, total: 1.98 ms
Wall time: 1.27 ms


In [ ]:
%%time
gibs_it_sanic(100)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
Iteration: 38


In [181]:
%lprun -f gibs_it_sanic gibs_it_sanic(10)

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10


In [75]:

to_sort = []
for i in range(len(documentToTopic)):
    to_sort.append((documentToTopic[i][0], i))

to_sort.sort(key=lambda t: t[0], reverse=True)
result1 = df.content[to_sort[0][1]]
result2 = df.content[to_sort[1][1]]